# VESIcal at vVMSG 2022
Simon Matthews (University of Iceland) & Penny Wieser (Oregon State University)
## 02 - Introduction to VESIcal and Volatile Solubility Calculations
The first thing we must do is import the python tools (or packages and modules) that we will use for the calculations, including VESIcal. 

In [ ]:
# NumPy provides tools for doing numerical calculations
import numpy as np

# VESIcal provides the tools for performing volatile solubility calculations
import VESIcal as v


the code `as v` means we can access all of VESIcals functions using the variable name `v`.

If you have installed python on your own machine from [anaconda](anaconda.com) most of these packages will already be installed, but you would need to install the ENKI thermoengine and VESIcal. Fortunately, VESIcal comes pre-installed on the ENKI server.

Note that each notebook has no knowledge of what you did in another notebook, and if you restart the server it will have no knowledge of the code you ran in the past (except what its output was). This means we have to import the packages in every notebook we run.

### Exercise 02.01 - Calculate how much CO$_2$ and H$_2$O will be dissolved in a magma
The aim of this exercise is to explore what the different models predict for the dissolved concentrations of H$_2$O and CO$_2$ when the magma is in equilibrium with a H$_2$O-CO$_2$-vapour, with a chosen proportion of H$_2$O vs CO$_2$ and a chosen pressure.

First we must create a sample composition, using the `Sample` class built into VESIcal. The example here is a melt inclusion composition from Kilauea (Wieser et al., 2021):

In [ ]:
kilauea = v.Sample({'SiO2':  48.42,
                    'TiO2':   2.45,
                    'Al2O3': 11.90,
                    'Fe2O3':  0.00,
                    'FeO':   11.33,
                    'MgO':   12.51,
                    'CaO':   10.02,
                    'Na2O':   2.10,
                    'K2O':    0.45,
                    'P2O5':   0.30,
                    })

**Note that VESIcal will assume your composition is given in wt%, unless you tell it otherwise**
There is no output this time- we are just asking the notebook to store information in a particular format for us.

Now we can calculate how much H$_2$O and CO$_2$ will be stored in the melt under a certain set of conditions, using the MagmaSat model:

In [ ]:
# Use variables to store our choices of calculation parameter.
P = 1000 # bars
T = 1200 # degrees C
fluid_composition = 0.5 # Mole fraction of H2O

# Create a calculation for dissolved volatiles, using the sample we defined
# and the parameters above.
calc = v.calculate_dissolved_volatiles(sample=kilauea, 
                                       pressure=P, 
                                       temperature=T, 
                                       X_fluid=fluid_composition)

# Print out the result of the calculation
calc.result

The results are given in wt%.

We can now try doing the same calculation with the other models. Can't remember what models are available? No problem- we can find out!

In [ ]:
v.get_model_names()

We can change the model used in the calculation by adding an extra *argument* when we create the calculation:

In [ ]:
# Create a calculation for dissolved volatiles, using the sample we defined
# and the parameters above.
calc = v.calculate_dissolved_volatiles(sample=kilauea, 
                                       pressure=P, 
                                       temperature=T, 
                                       X_fluid=fluid_composition,
                                       model='IaconoMarziano') # HERE'S THE EXTRA ARGUMENT!

# Print out the result of the calculation
calc.result

Notice that it uses the same variables we defined in the cell above. When we give VESIcal the model name it is enclosed in quotation marks- this tells python that we're giving it letters that mean something (a string), rather than a variable name. If you are curious what difference this makes, try re-running the code without the quotation marks!

Looking at the calculation result we see that there is quite a big difference in the predicted solubilities.

### Exercise 02.02 - Play with the models!
Try running some more calculations, but changing the parameters and the model you are using. You can either copy the code given above and put it into new cells below, or you can just edit and re-run the cells above. How much do the models seem to agree?

### Exercise 02.03 - Using measured CO$_2$ and H$_2$O to get saturation pressure
Very often we are interested in the reverse calculation- given the dissolved H$_2$O and CO$_2$ concentrations in a melt inclusion, at what pressure would this be in equilibrium with a vapour.

VESIcal has a built in method for doing this too, and you can use it in a very similar way to calculating the dissolved H$_2$O and CO$_2$.

This time we need to define a sample composition that includes H$_2$O and CO$_2$:

In [ ]:
kilauea = v.Sample({'SiO2':  48.42,
                    'TiO2':   2.45,
                    'Al2O3': 11.90,
                    'Fe2O3':  0.00,
                    'FeO':   11.33,
                    'MgO':   12.51,
                    'CaO':   10.02,
                    'Na2O':   2.10,
                    'K2O':    0.45,
                    'P2O5':   0.30,
                    'H2O':    0.17,
                    'CO2':    791.0/1e4 # Convert from ppmw to wt%
                    })

# We still need to define a temperature:
T = 1200 # degrees C

In [ ]:
calcSatP = v.calculate_saturation_pressure(sample=kilauea,
                                           temperature=T)
calcSatP.result

As before, the returned pressure is in bars. Since we didn't specify the model, VESIcal used MagmaSat.

How does this estimate vary if we switch to a different model? Well we could copy and paste code and update it each time (as I suggested you do in the previous exercise). Or we could use another feature of python- and automate the repetitive calculation:

In [ ]:
# These are the models we will repeat the calculation with:
models_to_use = ['MagmaSat', 'IaconoMarziano', 'Dixon', 'ShishkinaIdealMixing']

# For each of the models in the list, run this code (substituting the variable model each time)
for model in models_to_use:
    calcSatP = v.calculate_saturation_pressure(sample=kilauea,
                                               temperature=T,
                                               model=model)
    
    # Print a formatted statement with the result from each model
    print("Saturation Pressure from {} is {:.1f} bars.".format(model, calcSatP.result))

That cell may have reported an error in finding the Saturation pressure. Usually this is an important error to pay attention to. In this case, it is a bug that I discovered while writing this workshop! The result is correct, but I only know that after further investigation. Hopefully this won't be an issue for your calculations! And if it is- let us know and we will fix it ASAP!

We can also calculate the fluid composition that would be in equilibrium the sample. I have adjusted the loop above to use a different calculation:

In [ ]:
# These are the models we will repeat the calculation with:
models_to_use = ['MagmaSat', 'IaconoMarziano', 'Dixon', 'ShishkinaIdealMixing']

# For each of the models in the list, run this code (substituting the variable model each time)
for model in models_to_use:
    calcSatP = v.calculate_equilibrium_fluid_comp(sample=kilauea, # THIS IS THE LINE I CHANGED
                                                  temperature=T,
                                                  model=model)
    
    # Print a formatted statement with the result from each model
    print("The {} model predicts a H2O mole fraction of {:.3f} and a CO2 fraction " 
          "of {:.3f}.".format(model, calcSatP.result['H2O'], calcSatP.result['CO2']))

In [ ]:
# These are the models we will repeat the calculation with:
models_to_use = ['MagmaSat', 'IaconoMarziano', 'Dixon', 'ShishkinaIdealMixing']

# For each of the models in the list, run this code (substituting the variable model each time)
for model in models_to_use:
    calcSatP = v.calculate_equilibrium_fluid_comp(sample=kilauea,
                                                  temperature=T,
                                                  model=model)
    
    # Print a formatted statement with the result from each model
    print("Saturation Pressure from {} is {} bars.".format(model, calcSatP.result))